<a href="https://colab.research.google.com/github/fultramari/Gerenciamento-e-interoperabilidade-de-Redes/blob/main/PLN_AtividadeExtra2_Analisando_o_myanimelist_via_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PARTE 1**: Instalando o framework da gemini, utilizando a chave criada e escolhendo algum dos modelos da LLM disponíveis.

In [ ]:
#Instalando o framework
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key = GOOGLE_API_KEY)


In [ ]:
# Lista de Modelos

for modelo in genai.list_models():
  if 'generateContent' in modelo.supported_generation_methods:
    print(modelo.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


In [ ]:
#Modelo escolhido
modelo = genai.GenerativeModel("gemini-1.5-pro")

**PARTE 2** : Extraindo o conteúdo da url

In [ ]:
import requests

# Headers necessários pra conseguir acessar o site simulando um acesso pelo navegador
# O header "Accept-Language" foi alterado para pt-BR para que os resultados fiquem em português

HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "pt-BR,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

url = 'https://myanimelist.net/topanime.php'

# faz um request http para acessar o site
resposta = requests.get(url, headers=HEADERS)

# Verifica se a solicitação foi bem-sucedida
if resposta.status_code == 200:
  print('Conexão bem-sucedida')


Conexão bem-sucedida


In [ ]:
from bs4 import BeautifulSoup
import json

# cria um objeto BeautifulSoup para parsear o HTML utilizando a resposta do request anterior
soup = BeautifulSoup(resposta.content, 'html.parser')
texto_visivel = soup.get_text(separator="\n", strip=True)

 # Limitar o tamanho do texto, se necessário
limite_caracteres = 1000000  # Ajuste conforme necessário
texto_visivel_truncado = texto_visivel[:limite_caracteres]
print(texto_visivel_truncado)

Top Anime - MyAnimeList.net
MyAnimeList.net
Hide Ads
Login
Sign Up
All
Anime
Manga
Characters
People
Companies
Manga Store
News
Featured Articles
Forum
Clubs
Users
View all results for
${ keyword }
Anime
Anime Search
Top Anime
Seasonal Anime
Videos
Reviews
Recommendations
2024 Challenge
Fantasy Anime League
Manga
Manga Search
Top Manga
Manga Store
Reviews
Recommendations
2024 Challenge
Community
Interest Stacks
Forums
Clubs
Blogs
Users
Industry
News
Featured Articles
People
Characters
Companies
MAL×Japan
Watch
Episode Videos
Anime Trailers
Read
Manga Store
Help
About
Support
Advertising
FAQ
Report
Staff
MAL Supporter
Top Anime
Top
>
Anime
>
Top Anime
All Anime
Top Airing
Top Upcoming
Top TV Series
Top Movies
Top OVAs
Top ONAs
Top Specials
Most Popular
Most Favorited
Next 50
Top Anime Series
Updated twice a day. (
How do we rank shows?
)
Rank
Title
Score
Your Score
Status
1
Sousou no Frieren
TV (28 eps)
Sep 2023 - Mar 2024
845,771 members
9.34
N/A
Add to My List
2
Fullmetal Alchemist: B

PARTE 3: Fornecendo as informações para a LLM extrair os dados convenientes.

In [ ]:
#Função criada para uma melhor visualização da resposta da LLM
def extract_content(response):
  """Extracts the content from the given GenerateContentResponse."""

  # Convert the GenerateContentResponse to a dictionary
  response_dict = response.to_dict()
  content = response_dict['candidates'][0]['content']['parts'][0]['text']
  return content

In [ ]:
import google.generativeai as genai
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

# Criar o prompt para o modelo Gemini
prompt = f"Extraia as informações mais relevantes da seguinte página:\n\n{texto_visivel_truncado}\n\n"

# Gerar texto usando o modelo Gemini
resposta = modelo.generate_content(prompt)
# Processar a resposta do Gemini


In [ ]:
print(extract_content(resposta))

## Informações relevantes da página "Top Anime - MyAnimeList.net":

**Título da Página:** Top Anime - MyAnimeList.net

**Funcionalidade:**  A página lista os animes mais bem avaliados no site MyAnimeList.net.

**Critério de Ranking:** A página não descreve exatamente a fórmula de pontuação, mas informa que o ranking é atualizado duas vezes ao dia e oferece um link para entender melhor como os animes são classificados: "(How do we rank shows?)".

**Informações Exibidas por Anime:**

* **Posição no Ranking:** Um número indica a posição do anime na lista.
* **Título:** Nome do anime.
* **Formato:** Informa se é um especial de TV, filme, OVA, ONA, etc. e a quantidade de episódios.
* **Período de Exibição:**  Indica quando o anime foi ao ar.
* **Número de Membros:** Quantidade de usuários que marcaram o anime em suas listas.
* **Pontuação:** Nota média do anime, variando de 1 a 10.
* **Sua Pontuação:** Campo para o usuário avaliar o anime, caso esteja logado.
* **Botão "Add to My List":** P

Passo 4: Sabendo que a página lista os animes de maior classificação, vamos extrair essas informações e armazenar em uma planilha.

In [ ]:
resposta2 = modelo.generate_content(f"A partir do texto {texto_visivel_truncado}, faça um dataframe no formato CSV Posição no ranking  ,Nome do anime , Período de transmissão , Avaliação.Se não conseguir extrair informação de alguma das colunas, ignore-a. Não quero nenhuma informação além disso, para ser possível copiar em uma variável e manipular.")
print(resposta2)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Posicao no ranking,Nome do anime,Periodo de transmissao,Avaliacao\n1,Sousou no Frieren,Sep 2023 - Mar 2024,9.34\n2,Fullmetal Alchemist: Brotherhood,Apr 2009 - Jul 2010,9.09\n3,Steins;Gate,Apr 2011 - Sep 2011,9.07\n4,Gintama\u00b0,Apr 2015 - Mar 2016,9.06\n5,Shingeki no Kyojin Season 3 Part 2,Apr 2019 - Jul 2019,9.05\n6,Gintama: The Final,Jan 2021 - Jan 2021,9.04\n7,Gintama',Apr 2011 - Mar 2012,9.03\n8,Hunter x Hunter (2011),Oct 2011 - Sep 2014,9.03\n9,Gintama': Enchousen,Oct 2012 - Mar 2013,9.02\n10,Bleach: Sennen Kessen-hen,Oct 2022 - Dec 2022,9.01\n11,Ginga Eiyuu Densetsu,Jan 1988 - Mar 1997,9.01\n12,Kaguya-sama wa Kokurasetai: Ultra Romantic,Apr 2022 - Jun 2022,9.00\n13,Gintama.,Jan 2017 - Mar 2017,8.98\n14,Fruits Basket: The Final,Apr 2021 - Jun 2021,8.9

In [ ]:
#Extraindo a tabela...
content = extract_content(resposta2)
print(content)

Posicao no ranking,Nome do anime,Periodo de transmissao,Avaliacao
1,Sousou no Frieren,Sep 2023 - Mar 2024,9.34
2,Fullmetal Alchemist: Brotherhood,Apr 2009 - Jul 2010,9.09
3,Steins;Gate,Apr 2011 - Sep 2011,9.07
4,Gintama°,Apr 2015 - Mar 2016,9.06
5,Shingeki no Kyojin Season 3 Part 2,Apr 2019 - Jul 2019,9.05
6,Gintama: The Final,Jan 2021 - Jan 2021,9.04
7,Gintama',Apr 2011 - Mar 2012,9.03
8,Hunter x Hunter (2011),Oct 2011 - Sep 2014,9.03
9,Gintama': Enchousen,Oct 2012 - Mar 2013,9.02
10,Bleach: Sennen Kessen-hen,Oct 2022 - Dec 2022,9.01
11,Ginga Eiyuu Densetsu,Jan 1988 - Mar 1997,9.01
12,Kaguya-sama wa Kokurasetai: Ultra Romantic,Apr 2022 - Jun 2022,9.00
13,Gintama.,Jan 2017 - Mar 2017,8.98
14,Fruits Basket: The Final,Apr 2021 - Jun 2021,8.97
15,Monogatari Series: Off & Monster Season,Jul 2024 - ,8.97
16,Gintama,Apr 2006 - Mar 2010,8.94
17,Clannad: After Story,Oct 2008 - Mar 2009,8.93
18,Koe no Katachi,Sep 2016 - Sep 2016,8.93
19,3-gatsu no Lion 2nd Season,Oct 2017 - Mar 2018,8.91
20,Cod

In [ ]:
import pandas as pd
from io import StringIO

# Converter a string 'content' para um dataframe
df = pd.read_csv(StringIO(content))

# Salvar o dataframe em uma planilha Excel
df.to_excel('ranking_animes.xlsx', index=False)

print("Planilha criada com sucesso!")


Planilha criada com sucesso!


In [ ]:
# Uma amostra da planilha
df

,Posicao no ranking,Nome do anime,Periodo de transmissao,Avaliacao
0,1,Sousou no Frieren,Sep 2023 - Mar 2024,9.34
1,2,Fullmetal Alchemist: Brotherhood,Apr 2009 - Jul 2010,9.09
2,3,Steins;Gate,Apr 2011 - Sep 2011,9.07
3,4,Gintama°,Apr 2015 - Mar 2016,9.06
4,5,Shingeki no Kyojin Season 3 Part 2,Apr 2019 - Jul 2019,9.05
5,6,Gintama: The Final,Jan 2021 - Jan 2021,9.04
6,7,Gintama',Apr 2011 - Mar 2012,9.03
7,8,Hunter x Hunter (2011),Oct 2011 - Sep 2014,9.03
8,9,Gintama': Enchousen,Oct 2012 - Mar 2013,9.02
9,10,Bleach: Sennen Kessen-hen,Oct 2022 - Dec 2022,9.01
